In [1]:
import numpy as np
import pandas as pd
from pysrc.optimization import gams, gurobi
from pysrc.sampling import baseline
from pysrc.services.data_service import load_site_data
from pysrc.analysis import value_decomposition, transfers_decomposition

/Users/ph2696/Projects/project-amazon/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model scenario

In [2]:
opt = "gurobi"
pee = 7.6
pa = 41.11
sitenum = 1043
T = 200

In [3]:
# Load site data
(
    zbar_2017,
    z_2017,
    forest_area_2017,
    _,
    _,
    _,
    _,
) = load_site_data(sitenum)

# Set productivity parameters using baseline mean
baseline_fit = baseline.sample(
    num_sites=sitenum,
    iter_sampling=10**4,
    chains=5,
    seed=1,
)

theta = baseline_fit.stan_variable("theta").mean(axis=0)
gamma = baseline_fit.stan_variable("gamma").mean(axis=0)

# Computing carbon absorbed in start period
x0_vals = gamma * forest_area_2017

Data successfully loaded from /Users/ph2696/Projects/project-amazon/data/hmc


22:10:45 - cmdstanpy - INFO - compiling stan file /Users/ph2696/Projects/project-amazon/stan_model/baseline.stan to exe file /Users/ph2696/Projects/project-amazon/stan_model/baseline


Data successfully loaded from /Users/ph2696/Projects/project-amazon/data/hmc


22:10:50 - cmdstanpy - INFO - compiled model executable: /Users/ph2696/Projects/project-amazon/stan_model/baseline
22:10:55 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status





chain 1 |          | 00:00 Status








chain 1 |▏         | 00:01 Iteration:    1 / 10000 [  0%]  (Sampling)





chain 1 |▎         | 00:01 Iteration:  100 / 10000 [  1%]  (Sampling)





chain 1 |▎         | 00:02 Iteration:  200 / 10000 [  2%]  (Sampling)





chain 1 |▍         | 00:02 Iteration:  300 / 10000 [  3%]  (Sampling)





chain 1 |▌         | 00:03 Iteration:  400 / 10000 [  4%]  (Sampling)





chain 1 |▋         | 00:03 Iteration:  500 / 10000 [  5%]  (Sampling)





chain 1 |▋         | 00:04 Iteration:  600 / 10000 [  6%]  (Sampling)





chain 1 |▊         | 00:04 Iteration:  700 / 10000 [  7%]  (Sampling)





chain 1 |▉         | 00:05 Iteration:  800 / 10000 [  8%]  (Sampling)





chain 1 |▉         | 00:05 Iteration:  900 / 10000 [  9%]  (Sampling)







22:11:46 - cmdstanpy - INFO - CmdStan done processing.


In [4]:
# Choose optimizer
if opt == "gurobi":
    solve_planner_problem = gurobi.solve_planner_problem

elif opt == "gams":
    solve_planner_problem = gams.solve_planner_problem

else:
    raise ValueError("Optimizer must be one of ['gurobi', 'gams']")

In [5]:
VDs = []
Rs = []

for b in range(6):
    results = solve_planner_problem(
        T=T,
        theta=theta,
        gamma=gamma,
        x0=x0_vals,
        zbar=zbar_2017,
        z0=z_2017,
        pe=pee + b * 5,
        pa=pa,
    )
    v = value_decomposition(
        Z=results["Z"],
        X=results["X"],
        U=results["U"],
        V=results["V"],
        T=T,
        pee=pee,
        pa=pa,
        b=b * 5,
        theta=theta,
    )
    Rs.append(results)
    VDs.append(v)

Solving the optimization problem...
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-30
Read LP format model from file /var/folders/d1/k6sr3htd7fdgk614mmq8pwq00000gr/T/tmpu5i_ejgx.pyomo.lp
Reading time = 1.08 seconds
x1: 417400 rows, 834601 columns, 2083071 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 417400 rows, 834601 columns and 2083071 nonzeros
Model fingerprint: 0x8154808b
Model has 200 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e-04, 1e+03]
  QObjective range [3e+03, 2e+05]
  Bounds range     [3e-06, 1e+00]
  RHS range        [9e-11, 5e-01]
Presolve removed 208947 rows and 209294 columns
Presolve time: 0.33s
Presolved: 208453 rows, 625307 columns, 1249370 nonzeros
Presolved model has 200 quadratic objective terms
Ordering time: 0.01s

B

In [6]:
print(pd.DataFrame(VDs).round(2).to_latex(index=False))

\begin{tabular}{rrrrrrrr}
\toprule
pa & pee & b & total_AO & total_NT & total_FS & total_AC & total_PV \\
\midrule
41.110000 & 7.600000 & 0 & 372.860000 & 0.000000 & -139.750000 & 7.690000 & 225.420000 \\
41.110000 & 7.600000 & 5 & 133.260000 & 30.430000 & 46.260000 & 5.640000 & 204.310000 \\
41.110000 & 7.600000 & 10 & 57.720000 & 116.050000 & 88.200000 & 11.730000 & 250.240000 \\
41.110000 & 7.600000 & 15 & 33.290000 & 197.210000 & 99.920000 & 17.630000 & 312.780000 \\
41.110000 & 7.600000 & 20 & 23.600000 & 274.680000 & 104.380000 & 22.490000 & 380.160000 \\
41.110000 & 7.600000 & 25 & 18.690000 & 350.920000 & 106.680000 & 26.630000 & 449.670000 \\
\bottomrule
\end{tabular}



In [11]:
def transfers_decomposition(
    years,
    Z,
    X,
    Z_base,
    X_base,
    b,
    delta=0.02,
    kappa=2.094215255,
):
    # Compute change in X
    X_dot = np.diff(X, axis=0)
    X_dot_base = np.diff(X_base, axis=0)

    # Compute net captured emissions for base case
    results_NCE_base = [-kappa * Z_base[t + 1] + X_dot_base[t] for t in range(years)]
    total_NCE_base = np.sum(results_NCE_base)

    # Compute NCE
    results_NCE = [-kappa * Z[t + 1] + X_dot[t] for t in range(years)]
    total_NCE = np.sum(results_NCE)

    results_NT2 = [
        -b * (kappa * Z[t + 1] - X_dot[t]) / ((1 + delta) ** t) for t in range(years)
    ]
    total_NT2 = np.sum(results_NT2)

    total_EC = total_NT2 / (total_NCE - total_NCE_base)

    return {
        "b": b,
        "net captured emissions": total_NCE,
        "discounted net transfers": total_NT2,
        "discounted effective costs": total_EC,
    }

In [15]:
TDs = []

for b, results in enumerate(Rs):
    TDs.append(
        transfers_decomposition(
            years=15,
            Z=results["Z"],
            X=results["X"],
            Z_base=Rs[0]["Z"],
            X_base=Rs[0]["X"],
            b=b * 5,
        )
    )

/var/folders/d1/k6sr3htd7fdgk614mmq8pwq00000gr/T/ipykernel_3141/3267766411.py:28: RuntimeWarning: invalid value encountered in scalar divide
  total_EC = total_NT2 / (total_NCE - total_NCE_base)


In [16]:
pd.DataFrame(TDs)

,b,net captured emissions,discounted net transfers,discounted effective costs
0,0,-12.087285,0.000000,NaN
1,5,2.388011,9.905044,0.684272
2,10,5.178144,43.497013,2.519313
3,15,6.640458,83.957932,4.483078
4,20,7.550020,127.724877,6.504196
5,25,8.130624,172.504289,8.532252


In [17]:
print(pd.DataFrame(TDs).round(2).to_latex(index=False))

\begin{tabular}{rrrr}
\toprule
b & net captured emissions & discounted net transfers & discounted effective costs \\
\midrule
0 & -12.090000 & 0.000000 & NaN \\
5 & 2.390000 & 9.910000 & 0.680000 \\
10 & 5.180000 & 43.500000 & 2.520000 \\
15 & 6.640000 & 83.960000 & 4.480000 \\
20 & 7.550000 & 127.720000 & 6.500000 \\
25 & 8.130000 & 172.500000 & 8.530000 \\
\bottomrule
\end{tabular}

